# Electoral guardian

In [3]:
# Provide dependencies 
! pip freeze > requirements.txt

In [4]:
# Install Dependencies 
! pip install -r requirements.txt

INFO: pip is looking at multiple versions of sagemaker to determine which version is compatible with other requirements. This could take a while.
  Using cached rpds_py-0.24.0-cp312-cp312-win_amd64.whl.metadata (4.2 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached pyzmq-26.4.0-cp312-cp312-win_amd64.whl.metadata (6.0 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached pywin32-310-cp312-cp312-win_amd64.whl.metadata (9.4 kB)
INFO: pip is still looking at multiple versions of sagemaker to determine which version is compatible with other requirements. This could take a while.
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached pygments-2.19.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached pydantic_core-2.33.2-cp312-cp312-win_amd64.whl.metadata (6.9 kB)
  Usin

ERROR: Cannot install -r requirements.txt (line 24), packaging==25.0 and sagemaker==2.243.3 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [3]:
# pip installations
! pip install pyngrok langgraph opencv-python Pillow python-dotenv transformers flask langsmith torch

  Using cached torch-2.7.0-cp312-cp312-win_amd64.whl.metadata (29 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached setuptools-80.1.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.7.0-cp312-cp312-win_amd64.whl (212.5 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
Using cached setuptools-80.1.0-py3-none-any.whl (1.2 MB)

   ---------------------------------------- 0/5 [mpmath]
   ---------------------------------------- 0/5 [mpmath]
   ---------------------------------------- 0/5 [mpmath]
   ---------------------------------------- 0/5 [mpmath]
   ---------------------------------------- 0/5 [mpmath]
   ---------------------------------------- 0/5 [mpmath]
   ---------------------------------------- 0/5 [mpm

In [47]:
# Environment initialization
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
import requests
import os
import shutil
import cv2
from transformers import pipeline
from PIL import Image
import server
from dotenv import load_dotenv

load_dotenv()

# Load environment variables
instance_id = os.getenv('API_INSTANCE_ZAPI')
token = os.getenv('INSTANCE_TOKEN')
phone_number = os.getenv('PHONE_NUMBER')
client_token = os.getenv('CLIENT_TOKEN')

# State definition
class GuardianState(TypedDict):
    media: Optional[bytes]
    tipo: Optional[str]
    analysis_result: Optional[dict]
    reliability_index: Optional[int]
    answer: Optional[str]

# Node functions
def receive_input(state: GuardianState) -> GuardianState:
    """Receives video from user via WhatsApp and downloads it"""
    print("📥 Receiving video via Whatsapp")

    answer = (
        f"⚠️ Wait a minute, we are analysing your video..\n"
    )

    url = f"https://api.z-api.io/instances/{instance_id}/token/{token}/send-text"
    payload = {
        "phone": phone_number,  # user number
        "message": answer
    }
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0 Edg/124.0.2478.80",
        'client-token': client_token,
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=10)
    except requests.exceptions.RequestException as e:
        print(f"❌ Error sending message:. {e}")
        answer = "generate connection error with the remote server."

    last_video_file = "last_video.txt"

    # Parse the file and retrieve the most recent URL.
    try:
        with open(last_video_file, "r") as f:
            video_url = f.read().strip()
    except FileNotFoundError:
        raise ValueError("URL file not found.")

    if not video_url:
        raise ValueError("Video URL not provided.")

    print(f"🎯 URL captured: {video_url}")

    try:
        response = requests.get(video_url, timeout=10)
        if response.status_code != 200:
            raise Exception(f"Failed to download the video: {response.status_code}")
        
        print(f"✅ Video downloaded successfully.")

    except Exception as e:
        print(f"❌ Error downloading the video: {e}")
        raise e  # Here we let the error propagate for Graph to handle.

    # Detect media type based on extension
    if video_url.endswith((".mp4", ".mov", ".avi")):
        tipo = "video"
        file_path = "input_video.mp4"
    else:
        raise ValueError("❌ Unsupported media format.")

    # Save the media on disk
    with open(file_path, "wb") as f:
        f.write(response.content)

    print(f"✅ {tipo.capitalize()} saved as {file_path}")

    return {
        **state,
        "media": file_path, 
        "tipo": tipo
    }

def process_media(state: GuardianState) -> GuardianState:
    """Process the input received."""
    print(f"🧪 Processing {state['tipo']}...")

    frames_folder = "frames"
    os.makedirs(frames_folder, exist_ok=True)

    # Extract frames
    cap = cv2.VideoCapture(state["media"])
    frame_idx = 0
    saved_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % 30 == 0:
            frame_path = os.path.join(frames_folder, f"frame_{saved_idx}.jpg")
            # Save the frame and verify if there was valid
            print(f"Saving frame {saved_idx} how {frame_path}")
            cv2.imwrite(frame_path, frame)
            saved_idx += 1
        frame_idx += 1
    cap.release()

    frames = os.listdir(frames_folder)
    if not frames:
        raise RuntimeError("No frames were extracted from the video.")

    #Deepfake detection
    deepfake_detector = pipeline("image-classification", model="Wvolf/ViT_Deepfake_Detection")

    fake_scores = []
    for frame_file in frames:
        frame_path = os.path.join(frames_folder, frame_file)

        # Resize the image to 224x224 before sending to the model
        frame_image = Image.open(frame_path)
        frame_image = frame_image.resize((224, 224))  
        frame_image.save(frame_path)  

        # Make the forecast
        preds = deepfake_detector(frame_path)
        print(f"Predictions for {frame_path}: {preds}")  

        # Check if the 'Fake' class exists and extract the score
        fake_score = 0
        for item in preds:
            if item['label'] == 'Fake':
                fake_score = item['score']
                break

        print(f"Deepfake score for {frame_path}: {fake_score}")
        fake_scores.append(fake_score)

    # Calculate the average deepfake score
    media_fake_score = sum(fake_scores) / len(fake_scores) if fake_scores else 0

    # Calibrate reliability index based on score
    reliability_index = int((1 - media_fake_score) ** 2 * 100)

    # Clear the frames after processing
    shutil.rmtree(frames_folder)

    explanation = (
    f"Average deepfake score: {reliability_index:.2f} | "
    )   

    return {
        **state,
        "analysis_result": {
            "explanation": explanation
        },
        "reliability_index": reliability_index
    }



def generate_answer(state: GuardianState) -> GuardianState:
    """Generates the answer for the user."""
    print("📤 Generating the answer for the user....")
    if 'reliability_index' not in state:
        print("❌ Key 'reliability_index' not found in state.")
        state['reliability_index'] = 0  # or any default value 

    if state['reliability_index'] >= 75:
        answer = (
            f"⚠️ Alert: possible manipulation detected..\n"
            f"Reliability Index: {state['reliability_index']} / 100\n"
            f"Details: {state['analysis_result']['explanation']}"
        )
    else:
        answer = (
            f"✅ No signs of manipulation.\n"
            f"Reliability Index: {state['reliability_index']} / 100"
        )

    
    # Send on WhatsApp 
    url = f"https://api.z-api.io/instances/{instance_id}/token/{token}/send-video"
    payload = {
        "phone": phone_number,  # user number
        "message": answer
    }
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0 Edg/124.0.2478.80",
        'client-token': client_token,
        'Content-Type': 'application/json'
    }
    
    try:
        response = requests.post(url, headers=headers, json=payload, timeout=10)
    except requests.exceptions.RequestException as e:
        print(f"❌ Error sending message:. {e}")
        answer = "generate connection error with the remote server."

    return {
        **state,
        "answer": answer
    }



In [48]:

# Graph construction
workflow = StateGraph(GuardianState)

workflow.add_node("receive_input", receive_input)
workflow.add_node("process_media", process_media)
workflow.add_node("generate_answer", generate_answer)

workflow.set_entry_point("receive_input")

workflow.add_edge("receive_input", "process_media")
workflow.add_edge("process_media", "generate_answer")
workflow.add_edge("generate_answer", END)

graph = workflow.compile()

# Creating an empty starting state
initial_state = {}

# Executes the graph
final_state = graph.invoke(initial_state)

print("\nFinal result:")
print(final_state["answer"])

📥 Receiving video via Whatsapp
🎯 URL captured: https://f004.backblazeb2.com/file/temp-file-download/instances/3E08C389DE8FD032C0B996E82C2DBF10/4A76F05E0E0ECA1291981D52095CBC6F/vuS4nUcmU4FEVr_ZQr6R9A==.mp4
✅ Video downloaded successfully.
✅ Video saved as input_video.mp4
🧪 Processing video...
Saving frame 0 how frames\frame_0.jpg
Saving frame 1 how frames\frame_1.jpg
Saving frame 2 how frames\frame_2.jpg
Saving frame 3 how frames\frame_3.jpg
Saving frame 4 how frames\frame_4.jpg
Saving frame 5 how frames\frame_5.jpg
Saving frame 6 how frames\frame_6.jpg
Saving frame 7 how frames\frame_7.jpg
Saving frame 8 how frames\frame_8.jpg
Saving frame 9 how frames\frame_9.jpg
Saving frame 10 how frames\frame_10.jpg
Saving frame 11 how frames\frame_11.jpg
Saving frame 12 how frames\frame_12.jpg
Saving frame 13 how frames\frame_13.jpg
Saving frame 14 how frames\frame_14.jpg
Saving frame 15 how frames\frame_15.jpg
Saving frame 16 how frames\frame_16.jpg
Saving frame 17 how frames\frame_17.jpg
Saving 

Device set to use cpu


Predictions for frames\frame_0.jpg: [{'label': 'Real', 'score': 0.9993258714675903}, {'label': 'Fake', 'score': 0.0006741755059920251}]
Deepfake score for frames\frame_0.jpg: 0.0006741755059920251
Predictions for frames\frame_1.jpg: [{'label': 'Real', 'score': 0.999024510383606}, {'label': 'Fake', 'score': 0.0009755033534020185}]
Deepfake score for frames\frame_1.jpg: 0.0009755033534020185
Predictions for frames\frame_10.jpg: [{'label': 'Real', 'score': 0.9990516304969788}, {'label': 'Fake', 'score': 0.0009483764879405499}]
Deepfake score for frames\frame_10.jpg: 0.0009483764879405499
Predictions for frames\frame_11.jpg: [{'label': 'Real', 'score': 0.999161958694458}, {'label': 'Fake', 'score': 0.0008380057406611741}]
Deepfake score for frames\frame_11.jpg: 0.0008380057406611741
Predictions for frames\frame_12.jpg: [{'label': 'Real', 'score': 0.9991752505302429}, {'label': 'Fake', 'score': 0.0008247693185694516}]
Deepfake score for frames\frame_12.jpg: 0.0008247693185694516
Predictions